In [ ]:
##Run this on 'http://14pyg.abmi.kr/user/hihipch/lab?'

In [4]:
import tarfile
tar = tarfile.open('/home/hihipch/brainCTclass.tar', "r:")
tar.extractall()
tar.close()

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import pydicom
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import keras
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.optimizers import Adadelta
from keras.backend import get_session

In [3]:
# For axialMIP
axialMIP_path = '/home/hihipch/brainCTclass/head-axial-MIP' 
axialMIP_list = []
for (path, dir, files) in os.walk(axialMIP_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.dcm':
            filePath = "{0}/{1}".format(path, filename)
            # print(filePath)
            axialMIP_list.append(filePath)

In [4]:
# For axialPostcon
axialPostcon_path = '/home/hihipch/brainCTclass/head-axial-postcon' 
axialPostcon_list = []
for (path, dir, files) in os.walk(axialPostcon_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.dcm':
            filePath = "{0}/{1}".format(path, filename)
            # print(filePath)
            axialPostcon_list.append(filePath)

In [5]:
# For axialPrecon
axialPrecon_path = '/home/hihipch/brainCTclass/head-axial-precon' 
axialPrecon_list = []
for (path, dir, files) in os.walk(axialPrecon_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.dcm':
            filePath = "{0}/{1}".format(path, filename)
            # print(filePath)
            axialPrecon_list.append(filePath)

In [6]:
# For coronalMIP
coronalMIP_path = '/home/hihipch/brainCTclass/head-coronal-MIP' 
coronalMIP_list = []
for (path, dir, files) in os.walk(coronalMIP_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.dcm':
            filePath = "{0}/{1}".format(path, filename)
            # print(filePath)
            coronalMIP_list.append(filePath)

In [7]:
# For coronalPrecon
coronalPrecon_path = '/home/hihipch/brainCTclass/head-coronal-precon' 
coronalPrecon_list = []
for (path, dir, files) in os.walk(coronalPrecon_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.dcm':
            filePath = "{0}/{1}".format(path, filename)
            # print(filePath)
            coronalPrecon_list.append(filePath)

In [8]:
# For sagittalMIP
sagittalMIP_path = '/home/hihipch/brainCTclass/head-sagittal-MIP' 
sagittalMIP_list = []
for (path, dir, files) in os.walk(sagittalMIP_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.dcm':
            filePath = "{0}/{1}".format(path, filename)
            # print(filePath)
            sagittalMIP_list.append(filePath)

In [9]:
# For topogram
topogram_path = '/home/hihipch/brainCTclass/head--topogram' 
topogram_list = []
for (path, dir, files) in os.walk(topogram_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.dcm':
            filePath = "{0}/{1}".format(path, filename)
            # print(filePath)
            topogram_list.append(filePath)

In [10]:
# For vrt
vrt_path = '/home/hihipch/brainCTclass/head--VRT' 
vrt_list = []
for (path, dir, files) in os.walk(vrt_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.dcm':
            filePath = "{0}/{1}".format(path, filename)
            # print(filePath)
            vrt_list.append(filePath)

In [9]:
print(len(axialMIP_list))
print(len(axialPostcon_list))
print(len(axialPrecon_list))
print(len(coronalMIP_list))
print(len(coronalPrecon_list))
print(len(sagittalMIP_list))

4677
1424
2143
1895
421
1993


In [10]:
###Create dataframe
# axialMIP=0
# axialPostcon=1
# axialPrecon=2
# coronalMIP=3
# coronalPrecon=4
# sagittalMIP=5


axialMIP_df = pd.DataFrame(data={'filename': axialMIP_list, 'label': [0 for i in range(len(axialMIP_list))]})
axialPostcon_df = pd.DataFrame(data={'filename': axialPostcon_list, 'label': [1 for i in range(len(axialPostcon_list))]})
axialPrecon_df = pd.DataFrame(data={'filename': axialPrecon_list, 'label': [2 for i in range(len(axialPrecon_list))]})
coronalMIP_df = pd.DataFrame(data={'filename': coronalMIP_list, 'label': [3 for i in range(len(coronalMIP_list))]})
coronalPrecon_df = pd.DataFrame(data={'filename': coronalPrecon_list, 'label': [4 for i in range(len(coronalPrecon_list))]})
sagittalMIP_df = pd.DataFrame(data={'filename': sagittalMIP_list, 'label': [5 for i in range(len(sagittalMIP_list))]})

df=axialMIP_df.append([axialPostcon_df, axialPrecon_df, coronalMIP_df, coronalPrecon_df, sagittalMIP_df])
df.columns=['filename', 'label']
df=df.sample(frac=1)
df.reset_index(drop=True,inplace=True)

In [11]:
### One hot encoding of label
y_train = np.array(df['label'].tolist())
y_train = keras.utils.np_utils.to_categorical(y_train)

In [12]:
### Resizing DICOM files in the df
x_train=[]
imageid=[]

for j in df['filename'].values:
    metadata=pydicom.filereader.dcmread(j)
    imageid.append(metadata.SOPInstanceUID)
    pixeldata=metadata.pixel_array
    pixeldata=cv2.resize(pixeldata, (224, 224))
    pixeldata[pixeldata < 0] = 0
    mean, std = pixeldata.mean(), pixeldata.std()
    regularized = (np.clip((pixeldata-mean)/std, -2.0, 2.0)+2.0)/4.0*255
    pixeldata = regularized.astype(np.uint8)
    try:
        backtorgb = cv2.cvtColor(pixeldata,cv2.COLOR_GRAY2RGB)
        mean, std = backtorgb.mean(), backtorgb.std()
        regularized = (np.clip((backtorgb-mean)/std, -2.0, 2.0)+2.0)/4.0
        x_train.append(regularized)
    except:
        mean, std = pixeldata.mean(), pixeldata.std()
        regularized = (np.clip((backtorgb-mean)/std, -2.0, 2.0)+2.0)/4.0
        x_train.append(regularized)
x_train=np.array(x_train)

In [13]:
x_train.shape, y_train.shape

((12553, 224, 224, 3), (12553, 6))

In [14]:
x_val=x_train[11000:]
y_val=y_train[11000:]

In [15]:
x_train=x_train[:11000]
y_train=y_train[:11000]

In [16]:
from keras.applications import ResNet50
from keras.layers import Dense, Input, Activation, Concatenate, Flatten
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adadelta, SGD, Nadam

num_classes=6
input_shape=(224, 224, 3)

model = ResNet50(include_top=False,
                            weights='imagenet',
                            input_shape=input_shape, pooling='avg')

x = Dense(num_classes, activation='softmax', name='predictions')(model.output)

model = Model(model.input, x)

sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [41]:
def cnn(size):
    KERNEL = (8,8)
    shape = (size[0], size[1], size[2])
    
    # Define Model
    model = Sequential()
    
    # Add Convolution 1
    model.add(Conv2D(filters=8, kernel_size=KERNEL, activation='relu', input_shape=shape))
    
    # Add Max Pooling
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    # Add Convolution 2
    model.add(Conv2D(filters=16, kernel_size=KERNEL, activation='relu', input_shape=shape))
    
    # Add Max Pooling
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    # Add Convolution 3
    model.add(Conv2D(filters=32, kernel_size=KERNEL, activation='relu', input_shape=shape))
    
    # Add Max Pooling
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(units=32, activation='relu'))
    
    # Add Output Layer
    model.add(Dropout(0.5))
    model.add(Dense(units=8, activation='softmax'))
    
    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer=Adadelta(), metrics=['accuracy'])
    model.summary()
    
    return model

model = cnn(size=regularized.shape[1:])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 121, 121, 8)       1544      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 60, 60, 8)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 60, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 53, 53, 16)        8208      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 26, 16)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 26, 26, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 19, 19, 32)       

In [ ]:
hist = model.fit(x_train, y_train, epochs=50, batch_size=16, validation_data=(x_val, y_val))

Train on 11000 samples, validate on 1553 samples
Epoch 1/50
11000/11000 [==============================] - 259s 24ms/step - loss: 0.1231 - accuracy: 0.9535 - val_loss: 0.9615 - val_accuracy: 0.6381
Epoch 2/50
11000/11000 [==============================] - 236s 21ms/step - loss: 0.0327 - accuracy: 0.9863 - val_loss: 0.0563 - val_accuracy: 0.9723
Epoch 3/50
11000/11000 [==============================] - 235s 21ms/step - loss: 0.0201 - accuracy: 0.9905 - val_loss: 0.0675 - val_accuracy: 0.9665
Epoch 4/50
11000/11000 [==============================] - 235s 21ms/step - loss: 0.0154 - accuracy: 0.9927 - val_loss: 0.0412 - val_accuracy: 0.9762
Epoch 5/50
11000/11000 [==============================] - 235s 21ms/step - loss: 0.0181 - accuracy: 0.9924 - val_loss: 0.0455 - val_accuracy: 0.9800
Epoch 6/50
11000/11000 [==============================] - 235s 21ms/step - loss: 0.0129 - accuracy: 0.9935 - val_loss: 0.0877 - val_accuracy: 0.9652
Epoch 7/50
11000/11000 [==============================] -

In [ ]:
# 테스트 데이터로 평가하기
score = model.evaluate(x_test, y_test, verbose=1)
print('loss=', score[0])
print('accuracy=', score[1])

In [ ]:
#use model
yhat_train=model.predict(regularized, batch_size=32)
yhat_val=model.predict(x_val, batch_size=32)
yhat_test=model.predict(x_test, batch_size=32)
print(len(yhat_train))
print(len(yhat_val))
print(len(yhat_test))
prediction=[]
for i in yhat_train:
    prediction.append(list(i).index(i.max()))
for i in yhat_val:
    prediction.append(list(i).index(i.max()))
for i in yhat_test:
    prediction.append(list(i).index(i.max()))

In [ ]:
seriesid=[]
for i in df['filename'].values:
    metadata=pydicom.filereader.dcmread(str(i))
    seriesid.append(metadata.SeriesInstanceUID)
filename=list(df['filename'])

In [ ]:
predictiondf=pd.DataFrame({'seriesid':seriesid, 'result': prediction, 'filepath':filename, 'label': list(df['label'])})
uniqueseriesid=[]
predictionvalue=[]
for i in list(set(predictiondf['seriesid'])):
    uniqueseriesid.append(i)
    strangedf=predictiondf.loc[predictiondf['seriesid'] == i]
    predictionvaluelist=list(strangedf['result'])
    predictionvalue.append(max(predictionvaluelist,key=predictionvaluelist.count))

In [ ]:
predictdf=pd.DataFrame({'seriesid':uniqueseriesid, 'predictionvalue':predictionvalue})
answerdf=predictiondf[['seriesid', 'label']].drop_duplicates()
scoredf=pd.merge(predictdf, answerdf)

In [ ]:
score=0
for i in range(len(scoredf)):
    if scoredf['predictionvalue'][i]==scoredf['label'][i]:
        score+=1
score

In [ ]:
#save model & weight
model_json = model.to_json()
with open('/home/hihipch/brainCTclass/model1.json', "w") as json_file : 
    json_file.write(model_json)
model.save_weights("/home/hihipch/brainCTclass/model1.h5")
print("Saved model to disk")